# What to expect

In notebook 2A (and 2B for your dataset) we:

- Had a look at the mapping results
- Combined reads per gene from all the samples in our experiment into a master dataframe and stored it as "analysis/Schistosoma_mansoni/star/ReadsPerGene.csv"
- Normalised the reads using PyDESeq2
- Visualised our results in a PCA plot

In this session, we will complete the analysis to find out which genes are differentially expressed. We will do this first for the example dataset <i>Schistosoma mansoni</i>, and then in notebook 3B you will repeat the process for your chosen dataset. 

For your dataset, you will also look at the GO terms and metabolic pathways associated with your differentially expressed genes.

# Set up
First, we need to import the required libraries and install pydeseq2 again. Since this time we also want to perform a statistical analysis, to find out which differences in gene expression are statistically significant, we also need to import DeseqStats.

In [ ]:
# import required libraries
import pandas as pd

#Install PyDESeq2 and import required classes
! pip install --quiet pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

We now need to create the dds object again, as we will use it for the next steps of the analysis.

As you might remember from workshop 2A, the <i>Schistosoma mansoni</i> dataset had samples from four different stages: 3 hr schistosomulum, 24 hr schistosomulum, cercarium and platyhelminth adult. We want to focus on comparing cercarium and 24hr schistosomulum.

<div class="alert alert-block alert-warning">
    
Create the counts and the metadata tables and restrict them so that they only contain the conditions we want to compare.

In [ ]:
# load the counts ("ReadsPerGene.csv") and metadata ("Metadata.csv") again - remember that we have to transpose the ReadsPerGene table to use it in PyDESeq2
counts = 
metadata = 

In [ ]:
# Have a look at the counts dataframe...
counts

In [ ]:
# ...and the metadata dataframe
metadata

<details>
<summary><i>Hints and tips</i></summary>
    
You could use the `.isin` method to filter rows, or `.loc` to select rows by their index. 

</details>

In [ ]:
# restrict to the 2 stages we want to compare: ["cercarium", "24hr schistosomulum"]

metadata_s = 

counts_s = 

Now that we have the counts and metadata, we can create the deseq2 dataset object again

In [ ]:
# create DESeq2 dataset object
dds = DeseqDataSet(
    counts=counts_s,
    metadata=metadata_s,
    design="stage",  # compare samples based on the developmental "stage"
    refit_cooks=True
)

# Differential Expression analysis

As we did in notebook 2A, we will apply the `deseq2` method to our dds object. Remember that this method normalises the data, estimates the dispersion and calculates the log fold change (LFC) based on the design factor.

In [ ]:
# Run DESeq2
dds.deseq2()

This time, however, we also want to perform a statistical analysis, to find out which differences in gene expression are statistically significant. To do that, we use the class `DeseqStats` on our dds object, and store the output in a new object called "stat_res"

In [ ]:
stat_res=DeseqStats(dds, contrast=["stage", "24 hr schistosomulum", "cercarium"])

Note that in the command we have to specify what we want to use in our pairwise comparison (`contrast`) 

- `stage` : this indicates that we are comparing samples based on their developmental stages
- `24 hr schistosomulum` and `cercarium`: this indicates the two stages we want to compare. The order in which we write them is important. With the code above, fold change will be calculated as Gene counts in 24 hr schistosomulum divided by Gene counts in cercarium

Now, we have to generate a summary of the statistical analysis contained in the "stat_res" object. To do that, we use the `summary` method.

In [ ]:
stat_res.summary()

Voila! This is the raw result of our differential expression analysis. We can find out more about the columns in these results on the [page for the DESeq2 gene-level differential expression workflow](https://master.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#building-the-results-table) , as the python module we are using was developed from the DESeq2 tool. 

Let's take a moment to make sure we understand the results.

<div class="alert alert-block alert-warning">

Please discuss the following in your group:

- baseMean: what is this value? How is it calculated?
  
- log2FoldChange: what is this value? Why do you think the fold change is calculated as log2? What does a positive log2Foldchange mean?
  
- lfcSE: what is this value?

- stat: what is this value?

- pvalue and padj: what is the difference between these two values?

You can add some notes from your discussion on this text cell

\

\

\

We will now store the results in a dataframe called "res", so we can work with the results.

To do this, we will apply the PyDESeq2 attribute `results_df` to our stat_res object

In [ ]:
res = stat_res.results_df
res

<div class="alert alert-block alert-success">
<b>Learning outcomes</b>

You should now...

- Know how to perform differential expression analysis using PyDESeq2
- Understand the results produced by PyDESeq2

# Cleaning and exploring the results

There are a couple of things we should check in our results dataframe

<div class="alert alert-block alert-warning">

Use `describe()` to get an overview of the res dataframe. We practised this in notebook 1. 

In [ ]:
#Add your code here


 <div class="alert alert-block alert-warning">
     
Discuss in your groups whether there is anything that does not look right

<details>
<summary><i>Hints and tips</i></summary>
    
    Think about what each column means and you will realise that there are three values that do not make sense

</details>

You can add some notes from your discussion on this text cell

\

\

\

Let's fix the first problem

In [ ]:
import numpy as np

# replace p-values and padj values of 0 with a very small number
res.loc[ res.pvalue == 0, "pvalue" ] = np.finfo(np.float64).tiny
res.loc[ res.padj == 0, "padj" ] = np.finfo(np.float64).tiny

In the code above:

- `.loc` is a pandas attribute used to access a group of rows and columns
- `res.pvalue == 0` is a boolean condition checking where the value in the column "pvalue" of the dataframe "res" is exactly zero
- `"pvalue"` indicates which column will be modified when the boolean condition is met. In other words, the code will target the "pvalue" column where its values are zero
- `np.finfo(np.float64)` finds the machine limits for the numpy data type float64 (decimal numbers)
- `.tiny` is an attribute of the limits obtained from np.finfo(np.float64) that provides the smallest positive float64 number that the system can handle

 <div class="alert alert-block alert-warning">
Let's check that it worked

In [ ]:
# Add the code here

Now that we got rid of the 0 values, let's save the dataframe in csv format in case we want to use it outside noteable

In [ ]:
# save to csv file
res.to_csv(f"analysis/Schistosoma_mansoni/24h_schistosomulum_vs_cercarium.full.csv")

Let's fix the second problem. 
The results for a gene that is very lowly expressed across all samples will probably not be very reliable, or very informative of a biological function. They might also make further visualisation skewed. To avoid this, we will remove genes with very low expression on average. A common (although arbitrary) threshold for this is a baseMean of 10.

<div class="alert alert-block alert-warning">

Fill in the gaps in the code below to remove results with baseMean<10 from the res dataframe. You practised how to do this in DExB2 class 8. 

In [ ]:
# Filter results with baseMean<10
res=res[...]
# Check that it has worked

Let's start exploring the results

<div class="alert alert-block alert-warning">

Find out and discuss in your group:

- How many genes are significantly differentially expressed?
- For how many of these significant genes is the fold change (FC) greater than 2 or less than 0.5? Store them in a new dataframe called "sigs", and save it as a csv file called "cercarium_vs_24h_schistosomulum.filtered"

In [ ]:
#Find out how many genes are significantly differentially expressed


Get a list of only those genes that have a fold change FC > 2 or FC < 0.5 and are significantly changed.

<details>
<summary><i>Hints and tips</i></summary>
    
- Significance here is considered to be a padj of \<0.05
- Don't forget that a log2\(2\) = 1 and log2\(0.5\) = -1

</details>

In [ ]:
# Get a list of only those genes that have a fold change FC > 2 or FC < 0.5 and are significantly changed
sigs = 
sigs

In [ ]:
# save it as a csv file
sigs.to_csv(f"analysis/Schistosoma_mansoni/24h_schistosomulum_vs_cercarium.filtered.csv")

<div class="alert alert-block alert-success">
<b>Learning outcomes</b>

You should now...

- Understand what to look for when cleaning differential expression analysis results
- Know how to explore and filter the results

# Visualisation

<figure>
    <img src="https://scienceparkstudygroup.github.io/rna-seq-lesson/img/volcano_plot.png" align="right" width="400">
</figure>

A common way to visualise differential expression analysis results is a volcano plot.
The advantage of this plot is that we can see at the same time the change in gene expression and the statistical significance of that change, for each gene. That makes it easy to identify genes for further analysis.
As shown in the figure on the right (taken from [this source](https://scienceparkstudygroup.github.io/rna-seq-lesson/06-differential-analysis/index.html#3-volcano-plot)), in a typical volcano plot:
* on the X axis we plot the log2 fold change in gene expression
* on the Y axis we plot the -log10(p-value). There are two reasons why we use -log10(p-value):

    (a) To help with visualisation, in the same way that the fold change in gene expression was calculated as log2 by PyDESeq2. 

    (b) If you remember previous stats lessons, p-values are a probability, and therefore are always between 0 and 1. The log10 of a number between 0 and 1 will always be negative. To make interpretation easier, we do "-"log10, so the result is always a positive number. For example: log10(0.5)=-0.301 ; -[log10(0.5)]=-[-0.301]=0.301.

Let's plot our results in a volcano plot. To do that, we will use matplotlib, which we import as "plt"

You used matplotlib in DExB2, for example in classes 5 and 6. If you need a quick refresher, have a look [here](https://matplotlib.org/stable/users/explain/quick_start.html)

In [ ]:
import matplotlib.pylab as plt

<div class="alert alert-block alert-warning">

Using matplotlib (plt), make a scatter plot that: 
* in the X axis plots the log2FoldChange values from the "res" dataframe 
* in the Y axis plots -log10 of the padj values from the "res" dataframe

In [ ]:
# First, create a new column in the dataframe res that contains the -log10(padj). 


In [ ]:
# Now, make the scatter plot using the code below
plt.scatter(x=...,y=..., s=1)

Let's make the volcano plot fancier by colouring dots depending on:
* whether a gene is up- or downregulated -> We will consider that a gene is up- or downregulated if its expression level at least doubles or halves between the two conditions
* whether the change is significant -> We will use padj<0.05

<div class="alert alert-block alert-warning">

Fill in the code below to colour the dots in the volcano plot

<details>
<summary><i>Hints and tips</i></summary>
    
In the `plt.scatter` command, `x` and `y` will need to be given data using `df['column_of_interest']`.
</details>

In [ ]:
# define which parameters determine if a gene is significantly up or down, and assign them to a new dataframe called up or down
down = res[(res['...']<=...)&(res['...']<...)]
up = res[(res['...']>=...)&(res['...']<...)]

# plot all the genes and label as non-significant
plt.scatter(x=...,y=..., s=1, label="Not significant")
# colour downregulated genes in blue
plt.scatter(x=...,y=...,s=1,label="Down-regulated",color="blue")
# colour upregulated genes in red
plt.scatter(x=...,y=...,s=1,label="Up-regulated",color="red")

<div class="alert alert-block alert-warning">

Complete the code below to improve the volcano plot further by adding:

* axes labels
* lines at the threshold values
* legend

And save it as a png image so you can use it later

In [ ]:
# define which parameters determine if a gene is significantly up or down, and assign them to a new dataframe called up or down
down = res[(res['...']<=...)&(res['...']<...)]
up = res[(res['...']>=...)&(res['...']<...)]

# plot all the genes and label as non-significant
plt.scatter(x=...,y=..., s=1, label="Not significant")
# colour downregulated genes in blue
plt.scatter(x=...,y=...,s=1,label="Down-regulated",color="blue")
# colour upregulated genes in red
plt.scatter(x=...,y=...,s=1,label="Up-regulated",color="red")

# Add axes labels
plt.xlabel("...")
plt.ylabel("...")

# Add threshold lines
plt.axvline(-2,color="grey",linestyle="--")
plt.axvline(2,color="grey",linestyle="--")
plt.axhline(2,color="grey",linestyle="--")

# Add a legend
plt.legend()

# Save as png
plt.savefig('...')

<div class="alert alert-block alert-success">
<b>Learning outcomes</b>

You should now...

- Know how to interpret a volcano plot
- Know how to create a volcano plot to visualise your results

# Extension - Making an interactive volcano plot

To explore our data, it would be very convenient if we could hover over a dot and get the gene name. Making that sort of interactive plot is possible using the python library [plotly](https://plotly.com/python/).

In [ ]:
# First, we install and import
! pip install plotly
import plotly.express as px

In [ ]:
# Plotly requires a slightly different input than matplotlib. In our dataframe res, we need to create a new column with the gene names


In [ ]:
# lets check that it worked


In [ ]:
# Now, we can use our new columns to make the volcano plot
fig = px.scatter(
    res,
    x='log2FoldChange',
    y='-log10 padj',
    hover_data=['log2FoldChange', 'padj','gene'],
    title='Volcano Plot 24h_schistosomulum_vs_cercarum'
)

# Show the plot
fig.show()